<div>
<center><img src="Flux-logo.svg" width="400"/>
</div>

# Welcome to the Flux Tutorial

> What is Flux Framework? 🤔️
 
Flux is a flexible framework for resource management, built for your site. The framework consists of a suite of projects, tools, and libraries that may be used to build site-custom resource managers for High Performance Computing centers and cloud environments. Flux is a next-generation resource manager and scheduler with many transformative capabilities like hierarchical scheduling and resource management (you can think of it as "fractal scheduling") and directed-graph based resource representations.

## I'm ready! How do I do this tutorial? 😁️

This tutorial is split into 3 chapters, each of which has a notebook:
* [Chapter 1: Getting started with Flux](./01_flux_tutorial.ipynb) (you're already here, it's this notebook!)
* [Chapter 2: Flux Plumbing](./02_flux_framework.ipynb)
* [Chapter 3: Lessons learned, next steps, and discussion](./03_flux_tutorial_conclusions.ipynb)

And if you have some extra time and interest, we have supplementary chapters to teach you about advanced (often experimental, or under development) features:

* [Supplementary Chapter 1: Using DYAD to accelerate distributed Deep Learning (DL) training](./supplementary/dyad/dyad_dlio.ipynb)

Let's get started! To provide some brief, added background on Flux and a bit more motivation for our tutorial, "Shift+Enter" the cell below to watch our YouTube video!

In [22]:
%%html
<iframe width="640" height="360" 
    src="https://www.youtube.com/embed/YIwt51dyXOE" 
    title="YouTube video player" 
    frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" 
    allowfullscreen>
</iframe>

<br>

# Getting started with Flux

The code and examples that this tutorial is based on can be found at [flux-framework/Tutorials](https://github.com/flux-framework/Tutorials/tree/master/2024-HPCIC-AWS). You can also find python examples in the `flux-workflow-examples` directory from the sidebar navigation in this JupyterLab instance. 

<div class="alert alert-block alert-info">
<span style="font-weight:600">Tip:</span> Did you know you can get help for flux or a flux command? For example, try "flux help" and "flux help jobs"
</div>

In [2]:
!flux help

Usage: flux [OPTIONS] COMMAND ARGS
  -h, --help             Display this message.
  -v, --verbose          Be verbose about environment and command search
  -V, --version          Display command and component versions
  -p, --parent           Set environment of parent instead of current instance

For general Flux documentation, please visit
    https://flux-framework.readthedocs.io

run and submit jobs, allocate resources
   submit             submit a job to a Flux instance
   run                run a Flux job interactively
   bulksubmit         submit jobs in bulk to a Flux instance
   alloc              allocate a new Flux instance for interactive use
   batch              submit a batch script to Flux

list and interact with jobs
   jobs               list jobs submitted to Flux
   top                display running Flux jobs
   pstree             display job hierarchies
   cancel             cancel one or more jobs
   pgrep/pkill        search or cancel matching jobs
   job      

In [3]:
!flux help jobs

FLUX-JOBS(1)                       flux-core                      FLUX-JOBS(1)

NAME
       flux-jobs - list jobs submitted to Flux

SYNOPSIS
       flux jobs [OPTIONS] [JOBID ...]

DESCRIPTION
       flux  jobs is used to list jobs run under Flux. By default only pending
       and running jobs for the current user are listed. Additional  jobs  and
       information  can  be  listed  using options listed below.  Alternately,
       specific job ids can be listed on the command line to only  list  those
       job IDs.

OPTIONS
       -a     List  jobs  in  all  states,  including  inactive jobs.  This is
              shorthand for --filter=pending,running,inactive.

       -A     List jobs of all users. This is shorthand for --user=all.

       -n, --no-header
              For default output, do not output column headers.

       -u, --user=[USERNAME|UID]
              List jobs for a specific username or userid. Specify all for all
              users.

       --name=[JOB NAME]
  

### What does the terminal prompt mean?
For cases when you need a terminal, we will <button data-commandLinker-command="terminal:open" data-name="flux" href="#">provide you with a button</button>! However, you can also select `File -> New -> Terminal` to open one on the fly. Let's next talk about flux instances.

## Flux Resources

When you are interacting with Flux, you will commonly want to know what resources are available to you. Flux uses [hwloc](https://github.com/open-mpi/hwloc) to detect the resources on each node and then to populate its resource graph.

You can access the topology information that Flux collects with the `flux resource` subcommand. Let's run `flux resource list` to see the resources available to us in this notebook:

In [5]:
!flux resource list

     STATE NNODES   NCORES    NGPUS NODELIST
      free      4       38        0 8660c254a8e[5,5,5,5]
 allocated      1        2        0 8660c254a8e5
      down      0        0        0 


Flux can also bootstrap its resource graph based on static input files, like in the case of a multi-user system instance setup by site administrators.  [More information on Flux's static resource configuration files](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/guide/admin.html#configuration).  Flux provides a more standard interface to listing available resources that works regardless of the resource input source: `flux resource`.

In [6]:
# To view status of resources
!flux resource status

     STATE UP NNODES NODELIST
     avail  ✔      4 8660c254a8e[5,5,5,5]


It might also be the case that you need to see queues. Here is how to do that:

In [7]:
!flux queue list

 DEFAULTTIME  TIMELIMIT     NNODES     NCORES      NGPUS
         inf        inf      0-inf      0-inf      0-inf


<br>

# Flux Commands 

Here are how Flux commands map to a scheduler you are likely familiar with, Slurm. A larger table with similar mappings for LSF, Moab, and Slurm can be [viewed here](https://hpc.llnl.gov/banks-jobs/running-jobs/batch-system-cross-reference-guides). For submitting jobs, you can use the `flux` `submit`, `run`, `bulksubmit`, `batch`, and `alloc` commands.

<table>
    <tr>
        <th>Operation</th>
        <th>Slurm</th>
        <th>Flux</th>
    </tr>
    <tr>
        <td>One-off run of a single job (blocking)</td>
        <td><code>srun</code></td>
        <td><code>flux run</code></td>
    </tr>
    <tr>
        <td>One-off run of a single job (interactive)</td>
        <td><code>srun --pty</code></td>
        <td><code>flux run -o pty.interactive</code></td>
    </tr>
    <tr>
        <td>One-off run of a single job (not blocking)</td>
        <td><code>NA</code></td>
        <td><code>flux submit</code></td>
    </tr>
    <tr>
        <td>Bulk submission of jobs (not blocking)</td>
        <td><code>NA</code></td>
        <td><code>flux bulksubmit</code></td>
    </tr>    
    <tr>
        <td>Watching jobs</td>
        <td><code>NA</code></td>
        <td><code>flux watch</code></td>
    </tr>
    <tr>
        <td>Querying the status of jobs</td>
        <td><code>squeue</code>/<code>scontrol show job <i>job_id</i></code></td>
        <td><code>flux jobs</code>/<code>flux job info <i>job_id</i></code></td>
    </tr>
    <tr>
        <td>Canceling running jobs</td>
        <td><code>scancel</code></td>
        <td><code>flux cancel</code></td>
    </tr>
    <tr>
        <td>Allocation for an interactive instance</td>
        <td><code>salloc</code></td>
        <td><code>flux alloc</code></td>
    </tr>
    <tr>
        <td>Submitting batch jobs</td>
        <td><code>sbatch</code></td>
        <td><code>flux batch</code></td>
    </tr>
</table>

## flux run

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Running a single job (blocking)
</div>

The `flux run` command submits a job to Flux (similar to `flux submit`) but then attaches to the job with `flux job attach`, printing the job's stdout/stderr to the terminal and exiting with the same exit code as the job. It's basically doing an interactive submit, because you will be able to watch the output in your terminal, and it will block your terminal until the job completes.

In [19]:
!flux run hostname

749a39b51885


The output from the previous command is the hostname (a container ID string in this case). If the job exits with a non-zero exit code this will be reported by `flux job attach` (occurs implicitly with `flux run`). For example, execute the following:

In [20]:
!flux run /bin/false

flux-job: task(s) exited with exit code 1


A job submitted with `run` can be canceled with two rapid `Cltr-C`s in succession, or a user can detach from the job with `Ctrl-C Ctrl-Z`. The user can then re-attach to the job by using `flux job attach JOBID`.

`flux submit` and `flux run` also support many other useful flags:

In [10]:
!flux run -n4 --label-io --time-limit=5s --env-remove=LD_LIBRARY_PATH hostname

3: 8660c254a8e5
2: 8660c254a8e5
1: 8660c254a8e5
0: 8660c254a8e5


In [11]:
# Uncomment and run this help command if you want to see all the flags for flux run
# !flux run --help

## flux submit

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Running a single job (not blocking)
</div>


The `flux submit` command submits a job to Flux and prints out the jobid.

In [1]:
# Let's peek at the help for flux submit!
!flux submit --help | head -n 15

usage: flux submit [OPTIONS...] COMMAND [ARGS...]

enqueue a job

positional arguments:
  command                     Job command and arguments

options:
  -h, --help                  show this help message and exit
  -q, --queue=NAME            Submit a job to a specific named queue
  -t, --time-limit=MIN|FSD    Time limit in minutes when no units provided,
                              otherwise in Flux standard duration, e.g. 30s,
                              2d, 1.5h
      --urgency=N             Set job urgency (0-31), hold=0, default=16,
                              expedite=31


In [2]:
!flux submit hostname

ƒckWM1ZXM


But how does one get output? To quickly see output (which will block the terminal if the job is still running) after a submit, you can do:

```bash
flux job attach $(flux job last)
```

To provide a custom path to an output or error file, you can provide `--out` and `--err`, respectively. Let's try those both now.

In [10]:
# What was the last job id again?
! flux job last

# Attach to the last job id that was submitted (will block if still running and stream output)
! flux job attach $(flux job last)

ƒckWM1ZXM
749a39b51885


In [16]:
# Now let's submit another one, and give it the same output and error file
! flux submit --out /tmp/hola-cola.txt --err /tmp/hola-cola.txt echo "Did a polar bear with a soft drink write this...?! 🐻‍❄️🥤️😎️ "

# Take a look!
! cat /tmp/hola-cola.txt

ƒfeTb2bBm
Did a polar bear with a soft drink write this...?! 🐻‍❄️🥤️😎️ 


`submit` supports common options like `--nnodes`, `--ntasks`, and `--cores-per-task`. There are short option equivalents (`-N`, `-n`, and `-c`, respectively) of these options as well. `--cores-per-task=1` is the default.

In [14]:
!flux submit -N1 -n2 sleep inf

ƒ3VqVSHr7q


## flux bulksubmit

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Submitting jobs in bulk (not blocking)
</div>

The `flux bulksubmit` command enqueues jobs based on a set of inputs which are substituted on the command line, similar to `xargs` and the GNU `parallel` utility, except the jobs have access to the resources of an entire Flux instance instead of only the local system.

In [15]:
!flux bulksubmit --watch --wait echo {} ::: foo bar baz

ƒ3VqabmM3V
ƒ3VqabmM3W
ƒ3VqadFLKq
foo
bar
baz


### carbon copy

The `--cc` option (akin to "carbon copy") to `submit` makes repeated submission even easier via, `flux submit --cc=IDSET`:

In [16]:
!flux submit --cc=1-4 hostname

ƒ3VqhAnAU7
ƒ3VqhAnAU8
ƒ3VqhAnAU9
ƒ3VqhAnAUA


Try it in the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button> with a progress bar and jobs/s rate report: `flux submit --cc=1-100 --watch --progress --jps hostname`

Note that `--wait` is implied by `--watch`, meaning that when you are watching jobs, you are also waiting for them to finish. Here are some other carbon copy commands that are useful to try:

In [17]:
# Use flux carbon copy to submit identical jobs with different inputs
!flux submit --cc="1-2" echo "Hello I am job {cc}"

ƒ3Vqogq1L3
ƒ3Vqogq1L4


Here are some "carbon copy" jobs to try in the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button>:

```bash
# Use flux carbon copy to submit identical jobs with different inputs
flux submit --cc="1-10" echo "Hello I am job {cc}"

# Submits scripts myscript1.sh through myscript10.sh
flux submit --cc=0-6 flux-workflow-examples/bulksubmit/{cc}.sh

# Bypass the key value store and write output to file with jobid
flux submit --cc=1-10 --output=job-{{id}}.out echo "This is job {cc}"

# Use carbon copy to submit identical jobs with different inputs
flux bulksubmit --dry-run --cc={1} echo {0} ::: a b c ::: 0-1 0-3 0-7
```

Of course, Flux can launch more than just single-node, single-core jobs.  We can submit multiple heterogeneous jobs and Flux will co-schedule the jobs while also ensuring no oversubscription of resources (e.g., cores). Let's run the second example here, and add a clever trick to ask for output as we submit the jobs. This is a fun one, I promise!

In [18]:
! for jobid in $(flux submit --cc=0-6 /bin/bash flux-workflow-examples/bulksubmit/{cc}.sh); do flux job attach ${jobid}; done

Once upon a time... 📗️
There was a little duck 🦆️
Her name was pizzaquack 🍕️
She was very fond of cheese 🧀️
And running Flux 🌀️
And so she ran Flux, while she ate her cheese 😋️
And was so happy! The end. 🌈️


Note: in this tutorial, we cannot assume that the host you are running on has multiple cores, thus the examples below only vary the number of nodes per job.  Varying the `cores-per-task` is also possible on Flux when the underlying hardware supports it (e.g., a multi-core node). Let's run the middle example - it's a fun one, I promise!

In [18]:
!flux submit --nodes=2 --ntasks=2 --cores-per-task=1 --job-name simulation sleep inf
!flux submit --nodes=1 --ntasks=1 --cores-per-task=1 --job-name analysis sleep inf

ƒ3VqtrJWFh
ƒ3VqzNXq8B


## flux watch

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> 👀️ Watching jobs
</div>

Wouldn't it be cool to submit a job and then watch it? Well, yeah! We can do this now with flux watch. Let's run a fun example, and then watch the output. We have sleeps in here interspersed with echos only to show you the live action! 🥞️
Also note a nice trick - you can always use `flux job last` to get the last JOBID.
Here is an example (not runnable, as notebooks don't support environment variables) for getting and saving a job id:

```bash
flux submit hostname
JOBID=$(flux job last)
```

And then you could use the variable `$JOBID` in your subsequent script or interactions with Flux! So what makes `flux watch` different from `flux job attach`? Aside from the fact that `flux watch` is read-only, `flux watch` can watch many (or even all (`flux watch --all`) jobs at once!

In [19]:
!flux submit ./flux-workflow-examples/job-watch/job-watch.sh
!flux watch $(flux job last)

ƒ3Vr6FWywV
25 chocolate chip pancakes on the table... 25 chocolate chip pancakes! 🥞️
Eat a stack, for a snack, 15 chocolate chip pancakes on the table! 🥄️
15 chocolate chip pancakes on the table... 15 chocolate chip pancakes! 🥞️
Throw a stack... it makes a smack! 15 chocolate chip pancakes on the wall! 🥞️
You got some cleaning to do 🧽️


## flux jobs

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Querying the status of jobs
</div>

We can now list the jobs in the queue with `flux jobs` and we should see both jobs that we just submitted. Jobs that are instances are colored blue in output, red jobs are failed jobs, and green jobs are those that completed successfully. Note that the JupyterLab notebook may not display these colors. You will be able to see them in the terminal.

In [20]:
!flux jobs

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3VqzNXq8B jovyan   analysis    R      1      1   10.49s 8660c254a8e5
  ƒ3VqtrJWFh jovyan   simulation  R      2      2   10.71s 8660c254a8e[5,5]
  ƒ3VqVSHr7q jovyan   sleep       R      2      1   11.62s 8660c254a8e5
    ƒnyvM4Nb jovyan   sleep       R      2      1   5.269h 8660c254a8e5


You might also want to see "all" jobs with `-a`.

In [21]:
!flux jobs -a

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3VqzNXq8B jovyan   analysis    R      1      1   10.71s 8660c254a8e5
  ƒ3VqtrJWFh jovyan   simulation  R      2      2   10.92s 8660c254a8e[5,5]
  ƒ3VqVSHr7q jovyan   sleep       R      2      1   11.84s 8660c254a8e5
    ƒnyvM4Nb jovyan   sleep       R      2      1   5.269h 8660c254a8e5
  ƒ3Vr6FWywV jovyan   job-watch+ CD      1      1   10.03s 8660c254a8e5
  ƒ3Vqogq1L3 jovyan   echo       CD      1      1   0.015s 8660c254a8e5
  ƒ3Vqogq1L4 jovyan   echo       CD      1      1   0.014s 8660c254a8e5
  ƒ3VqhAnAUA jovyan   hostname   CD      1      1   0.060s 8660c254a8e5
  ƒ3VqhAnAU9 jovyan   hostname   CD      1      1   0.050s 8660c254a8e5
  ƒ3VqhAnAU8 jovyan   hostname   CD      1      1   0.047s 8660c254a8e5
  ƒ3VqhAnAU7 jovyan   hostname   CD      1      1   0.047s 8660c254a8e5
  ƒ3VqadFLKq jovyan   echo       CD      1      1   0.025s 8660c254a8e5
  ƒ3VqabmM3W jovyan   echo       CD      1      1   0.025s 8660c254a

## flux cancel

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Canceling running jobs
</div>

Since some of the jobs we see in the table above won't ever exit (and we didn't specify a timelimit), let's cancel them all now and free up the resources.

In [22]:
# This was previously flux cancelall -f
!flux cancel --all
!flux jobs

flux-cancel: Canceled 4 jobs (0 errors)
       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO


## flux alloc

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Allocation for an interactive instance
</div>

You might want to request an allocation for a set of resources (an allocation) and then attach to them interactively. This is the goal of flux alloc. Since we can't easily do that in a cell, try opening up the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button> and doing: 

```bash
# Look at the resources you have outside of the allocation
flux resource list

# Request an allocation with 2 "nodes" - a subset of what you have in total
flux alloc -N 2

# See the resources you are given
flux resource list

# You can exit from the allocation like this!
exit
```
When you want to automate this, submitting work to an allocation, you would use `flux batch`.

## flux batch

<div class="alert alert-block" style="background-color:skyblue">
<span style="font-weight:600">Description:</span> Submitting batch jobs
</div>

We can use the `flux batch` command to easily created nested flux instances.  When `flux batch` is invoked, Flux will automatically create a nested instance that spans the resources allocated to the job, and then Flux runs the batch script passed to `flux batch` on rank 0 of the nested instance. "Rank" refers to the rank of the Tree-Based Overlay Network (TBON) used by the [Flux brokers](https://flux-framework.readthedocs.io/projects/flux-core/en/latest/man1/flux-broker.html).

While a batch script is expected to launch parallel jobs using `flux run` or `flux submit` at this level, nothing prevents the script from further batching other sub-batch-jobs using the `flux batch` interface, if desired.

In [23]:
!flux batch --nslots=2 --cores-per-slot=1 --nodes=2 ./sleep_batch.sh
!flux batch --nslots=2 --cores-per-slot=1 --nodes=2 ./sleep_batch.sh

ƒ3Vw1mYfjD
ƒ3Vw6xW9wD


Take a quick look at [sleep_batch.sh](sleep_batch.sh) to see what we are about to run.

In [24]:
# Here we are submitting a job that generates output, and asking to write it to /tmp/cheese.txt
!flux submit --out /tmp/cheese.txt echo "Sweet dreams 🌚️ are made of cheese, who am I to diss a brie? 🧀️"

# This will show us JOBIDs
!flux jobs

# We can even see jobs in sub-instances with "-R" (for recursive)
!flux jobs -R

ƒ3VwC9Te9D
       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3Vw6xW9wD jovyan   ./sleep_b+  R      2      2   0.368s 8660c254a8e[5,5]
  ƒ3Vw1mYfjD jovyan   ./sleep_b+  R      2      2   0.572s 8660c254a8e[5,5]
       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3Vw6xW9wD jovyan   ./sleep_b+  R      2      2   0.536s 8660c254a8e[5,5]
  ƒ3Vw1mYfjD jovyan   ./sleep_b+  R      2      2   0.741s 8660c254a8e[5,5]

ƒ3Vw6xW9wD:

ƒ3Vw1mYfjD:


### `flux job`

Let's next inspect the last job we ran with `flux job info` and target the last job identifier with `flux job last`. 

In [25]:
# Note here we are using flux job last to see the last job id
# The "R" here asks for the resource spec
!flux job info $(flux job last) R

# When we attach it will direct us to our output file
!flux job attach $(flux job last)

# And we can look at the output file to see our expected output!
from IPython.display import Code
Code(filename='/tmp/cheese.txt', language='text')

{"version": 1, "execution": {"R_lite": [{"rank": "2", "children": {"core": "7"}}], "nodelist": ["8660c254a8e5"], "starttime": 1721520196, "expiration": 4875116178}}
0: stdout redirected to /tmp/cheese.txt
0: stderr redirected to /tmp/cheese.txt


Sweet dreams 🌚️ are made of cheese, who am I to diss a brie? 🧀️

We can again see a list all completed jobs with `flux jobs -a`:

In [26]:
!flux jobs -a

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3Vw6xW9wD jovyan   ./sleep_b+  R      2      2   0.998s 8660c254a8e[5,5]
  ƒ3Vw1mYfjD jovyan   ./sleep_b+  R      2      2   1.202s 8660c254a8e[5,5]
  ƒ3VwC9Te9D jovyan   echo       CD      1      1   0.014s 8660c254a8e5
    ƒnyvM4Nb jovyan   sleep      CA      2      1   5.269h 8660c254a8e5
  ƒ3VqVSHr7q jovyan   sleep      CA      2      1   12.04s 8660c254a8e5
  ƒ3VqzNXq8B jovyan   analysis   CA      1      1   10.91s 8660c254a8e5
  ƒ3VqtrJWFh jovyan   simulation CA      2      2   11.12s 8660c254a8e[5,5]
  ƒ3Vr6FWywV jovyan   job-watch+ CD      1      1   10.03s 8660c254a8e5
  ƒ3Vqogq1L3 jovyan   echo       CD      1      1   0.015s 8660c254a8e5
  ƒ3Vqogq1L4 jovyan   echo       CD      1      1   0.014s 8660c254a8e5
  ƒ3VqhAnAUA jovyan   hostname   CD      1      1   0.060s 8660c254a8e5
  ƒ3VqhAnAU9 jovyan   hostname   CD      1      1   0.050s 8660c254a8e5
  ƒ3VqhAnAU8 jovyan   hostname   CD      1      1   0.047s 8

To restrict the output to failed (i.e., jobs that exit with nonzero exit code, time out, or are canceled or killed) jobs, run:

In [27]:
!flux jobs -f failed

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
  ƒ3Vq5LFXNf jovyan   false       F      1      1   0.037s 8660c254a8e5
  ƒ2YnijmLwy jovyan   compute.py  F      1      1   0.031s 8660c254a8e5
  ƒ2YiqfxNdm jovyan   compute.py  F      1      1   0.012s 8660c254a8e5
  ƒ2YYgVHnyV jovyan   compute.py  F      1      1   0.062s 8660c254a8e5
  ƒ2YYE7Ja9d jovyan   compute.py  F      1      1   0.048s 8660c254a8e5
   ƒ3cZKNgsB jovyan   Hello I a+  F      1      1   0.014s 8660c254a8e5
   ƒ3cZKNgsA jovyan   Hello I a+  F      1      1   0.014s 8660c254a8e5
   ƒ3cZKNgs9 jovyan   Hello I a+  F      1      1   0.015s 8660c254a8e5
   ƒ3cZKNgsC jovyan   Hello I a+  F      1      1   0.012s 8660c254a8e5
    ƒhFVr6U7 jovyan   false       F      1      1   0.012s 8660c254a8e5


### flux submit from within a batch

Next open up [hello-batch.sh](hello-batch.sh) to see an example of using `flux batch` to submit jobs within the instance, and then wait for them to finish. This script is going to:

1. Create a flux instance with the top level resources you specify
2. Submit jobs to the scheduler controlled by the broker of that sub-instance
3. Run the four jobs, with `--flags=waitable` and `flux job wait --all` to wait for the output file
4. Within the batch script, you can add `--wait` or `--flags=waitable` to individual jobs, and use `flux queue drain` to wait for the queue to drain, _or_ `flux job wait --all` to wait for the jobs you flagged to finish. 

Note that when you submit a batch job, you'll get a job id back for the _batch job_, and usually when you look at the output of that with `flux job attach $jobid` you will see the output file(s) where the internal contents are written. Since we want to print the output file easily to the terminal, we are waiting for the batch job by adding the `--flags=waitable` and then waiting for it. Let's try to run our batch job now.

In [28]:
! flux batch --flags=waitable --out /tmp/flux-batch.out -N2 ./hello-batch.sh
! flux job wait
! cat /tmp/hello-batch-1.out
! cat /tmp/hello-batch-2.out
! cat /tmp/hello-batch-3.out
! cat /tmp/hello-batch-4.out

ƒ3VwkUsydR
ƒ3VwkUsydR
Hello job 1 from 8660c254a8e5 💛️
Hello job 2 from 8660c254a8e5 💚️
Hello job 3 from 8660c254a8e5 💙️
Hello job 4 from 8660c254a8e5 💜️


Each of `flux batch` and `flux alloc` hints at creating a Flux instance. How deep can we go into that rabbit hole, perhaps for jobs and workflows with nested logic or more orchestration complexity?

### The Flux Hierarchy 🍇️

One feature of the Flux Framework scheduler that is unique is its ability to submit jobs within instances, where an instance can be thought of as a level in a graph. Let's start with a basic image - this is what it might look like to submit to a scheduler that is not graph-based (left), where all jobs go to a central job queue or database. Note that our maximum job throughput is one job per second. The throughput is limited by the workload manager's ability to process a single job. We can improve upon this by simply adding another level, perhaps with three instances. For example, let's say we create a flux allocation or batch that has control of some number of child nodes. We might launch three new instances (each with its own scheduler and queue, right image) at that level two, and all of a sudden, we get a throughput of 1x3, or three jobs per second.

<table>
<tr>
    <td>
<img src="img/single-submit.png" style="float:left; margin-top:30px" width="350px">        
    </td>
    <td>
<img src="img/instance-submit.png" style="float:right; margin-top:-20px" width="550px">        
    </td>
    </tr>
</table>

All of a sudden, the throughout can increase exponentially because we are essentially submitting to different schedulers. The example above is not impressive, but our [learning guide](https://flux-framework.readthedocs.io/en/latest/guides/learning_guide.html#fully-hierarchical-resource-management-techniques) (Figure 10) has a beautiful example of how it can scale, done via an actual experiment. We were able to submit 500 jobs/second using only three levels, vs. close to 1 job/second with one level. For an interesting detail, you can vary the scheduler algorithm or topology within each sub-instance, meaning that you can do some fairly interesting things with scheduling work, and all without stressing the top level system instance. 

Now that we understand nested instances, let's look at another batch example that better uses them. Here we have two job scripts:

- [sub_job1.sh](sub_job1.sh): Is going to be run with `flux batch` and submit sub_job2.sh
- [sub_job2.sh](sub_job2.sh): Is going to be submitted by sub_job1.sh.

Take a look at each script to see how they work, and then submit it!

In [29]:
!flux batch -N1 ./sub_job1.sh

ƒ3Vxb9eQBy


And now that we've submitted, let's look at the hierarchy for all the jobs we just ran. Here is how to try flux pstree, which normally can show jobs in an instance, but it has limited functionality given we are in a notebook! So instead of just running the single command, let's add "-a" to indicate "show me ALL jobs."
More complex jobs and in a different environment would have deeper nesting. You can [see examples here](https://flux-framework.readthedocs.io/en/latest/jobs/hierarchies.html?h=pstree#flux-pstree-command).

In [30]:
!flux pstree -a

.
├── ./sub_job1.sh
├── ./sleep_batch.sh
│   └── sleep:R
├── ./sleep_batch.sh
│   └── sleep:R
├── ./hello-batch.sh:CD
├── 28*[echo:CD]
├── 2*[sleep:CA]
├── analysis:CA
├── simulation:CA
├── job-watch.sh:CD
├── 22*[hostname:CD]
├── 2*[false:F]
├── 200*[sleep:CD]
├── 4*[compute.py:F]
├── ./sub_job1.sh:CD
├── Hello I am job 3:F
├── Hello I am job 2:F
├── Hello I am job 1:F
└── Hello I am job 4:F


You can also try a more detailed view with `flux pstree -a -X`!

In [37]:
!flux exec -r all -x 0 flux archive extract --name myarchive --directory $(pwd) shared-file.txt

flux-archive: shared-file.txt: write: Attempt to overwrite existing file
flux-archive: shared-file.txt: write: Attempt to overwrite existing file
flux-archive: shared-file.txt: write: Attempt to overwrite existing file
[1-3]: Exit 1


<br>

# Process, Monitoring, and Job Utilities ⚙️

## flux exec 👊️

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Executing commands across ranks
</div>

Have you ever wanted a quick way to execute a command to all of your nodes in a flux instance? It might be to create a directory, or otherwise interact with a file. This can be hugely useful in environments where you don't have a shared filesystem, for example. This is a job for flux exec! Here is a toy example to execute the command to every rank (`-r all`) to print.

In [31]:
!flux exec -r all echo "Hello from a flux rank!"

Hello from a flux rank!
Hello from a flux rank!
Hello from a flux rank!
Hello from a flux rank!


You can also use `-x` to exclude ranks. For example, we often do custom actions on the main or "leader" rank, and just want to issue commands to the workers.

In [32]:
! flux exec -r all -x 0 echo "Hello from everyone except the lead (0) rank!"

Hello from everyone except the lead (0) rank!
Hello from everyone except the lead (0) rank!
Hello from everyone except the lead (0) rank!


Here is a similar example, but asking to execute only on rank 2, and to have it print the rank.

In [33]:
!flux exec -r 2 flux getattr rank 

2


And of course, we could do the same to print for all ranks! This is a derivative of the first example we showed you.

In [34]:
!flux exec flux getattr rank

0
3
2
1


You can imagine that `flux exec` is hugely useful in the context of batch jobs, and specific use cases with files, such as using `flux archive`, discussed next.

## flux archive 📚️

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Creating file and content archives to access later and between ranks
</div>

As Flux is used more in cloud environments, we might find ourselves in a situation where we have a cluster without a shared filesystem. The `flux archive` command helps with this situation. At a high level, `flux archive` allows us to save named pieces of data (e.g., files) to the Flux KVS for later retrieval.

When using `flux archive`, we first have to create an named archive. In the code below, we will create a text file and then save it into an archive using `flux archive`. Note that, for larger files, you can speed up the creation and extraction of archives by using the `--mmap` flag.

In [35]:
!echo "Sweet dreams 🌚️ are made of cheese, who am I to diss a brie? 🧀️" > shared-file.txt
!flux archive create --name myarchive --directory $(pwd) shared-file.txt

When we run this code, we are creating an archive in the leader broker. Now that the archive is created, we will want to extract its contents onto the other nodes of our cluster. To do this, we first need to ensure that the directory that we will extract into exists on those nodes. This can be done using `flux exec`. The `flux exec` command will execute a command on the nodes associated with specified brokers. Let's use `flux exec` to run `mkdir` on all the nodes of our cluster except the leader broker's node.

In [36]:
!flux exec -r all -x 0 mkdir -p $(pwd)

The flags provided to `flux exec` do the following:
* `-r all`: run across all brokers in the Flux instance
* `-x 0`: don't runn on broker 0 (i.e., the leader broker)

Now that the directory has been created on all our nodes, we can extract the archive onto those nodes by combining `flux exec` and `flux archive extract`.

Finally, when we're done with the archive, we can remove it with `flux archive remove`.

In [38]:
!flux archive remove --name myarchive

Finally, note that `flux archive` was named `flux filemap` in earlier versions of Flux.

## flux uptime

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Showing how long a flux instance has been running
</div>

Did someone say... [uptime](https://youtu.be/SYRlTISvjww?si=zDlvpWbBljUmZw_Q)? ☝️🕑️🕺️

Flux provides an `uptime` utility to display properties of the Flux instance such as state of the current instance, how long it has been running, its size and if scheduling is disabled or stopped. The output shows how long the instance has been up, the instance owner, the instance depth (depth in the Flux hierarchy), and the size of the instance (number of brokers).

In [39]:
!flux uptime

 00:03:20 run 5.3h,  owner jovyan,  depth 0,  size 4


## flux top 

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Showing a table of real-time Flux processes
</div>

Flux provides a feature-full version of `top` for nested Flux instances and jobs. In the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button> invoke `flux top` to see the "sleep" jobs. If they have already completed you can resubmit them. 

We recommend not running `flux top` in the notebook as it is not designed to display output from a command that runs continuously.

## flux pstree 

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Showing a flux process tree (and seeing nesting in instances)
</div>

In analogy to `top`, Flux provides `flux pstree`. Try it out in the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button> or here in the notebook.

## flux proxy

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Interacting with a job hierarchy
</div>

Flux proxy is used to route messages to and from a Flux instance. We can use `flux proxy` to connect to a running Flux instance and then submit more nested jobs inside it. From the <button data-commandLinker-command="terminal:open" data-name="flux" href="#">JupyterLab terminal</button> run the commands below!

```bash
# Outputs the JOBID
flux batch --nslots=2 --cores-per-slot=1 --nodes=2 ./sleep_batch.sh

# Put the JOBID into an environment variable
JOBID=$(flux job last)

# See the flux process tree
flux pstree -a

# Connect to the Flux instance corresponding to JOBID above
flux proxy ${JOBID}

# Note the depth is now 1 and the size is 2: we're one level deeper in a Flux hierarchy and we have only 2 brokers now.
flux uptime

# This instance has 2 "nodes" and 2 cores allocated to it
flux resource list
```

## flux queue

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Interacting with and inspecting Flux queues
</div>

Flux has a command for controlling the queue within the `job-manager`: `flux queue`.  This includes disabling job submission, re-enabling it, waiting for the queue to become idle or empty, and checking the queue status:

In [40]:
!flux queue disable "maintenance outage"
!flux queue enable
!flux queue -h

Job submission is disabled: maintenance outage
Job submission is enabled
usage: flux-queue [-h] {status,list,enable,disable,start,stop,drain,idle} ...

options:
  -h, --help            show this help message and exit

subcommands:

  {status,list,enable,disable,start,stop,drain,idle}


## flux getattr

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Getting attributes about your system and environment
</div>

Each Flux instance has a set of attributes that are set at startup that affect the operation of Flux, such as `rank`, `size`, and `local-uri` (the Unix socket usable for communicating with Flux).  Many of these attributes can be modified at runtime, such as `log-stderr-level` (1 logs only critical messages to stderr while 7 logs everything, including debug messages). Here is an example set that you might be interested in looking at:

In [41]:
!flux getattr rank
!flux getattr size
!flux getattr local-uri
!flux setattr log-stderr-level 3
!flux lsattr -v

0
4
local:///tmp/flux-iwjuLe/local-0
broker.boot-method                      simple
broker.critical-ranks                   0
broker.mapping                          [[0,1,4,1]]
broker.pid                              8
broker.quorum                           4
broker.quorum-timeout                   1m
broker.rc1_path                         /etc/flux/rc1
broker.rc3_path                         /etc/flux/rc3
broker.starttime                        1721501121.61
conf.shell_initrc                       /etc/flux/shell/initrc.lua
conf.shell_pluginpath                   /usr/lib/flux/shell/plugins
config.path                             -
content.backing-module                  content-sqlite
content.hash                            sha1
hostlist                                8660c254a8e[5,5,5,5]
instance-level                          0
jobid                                   -
local-uri                               local:///tmp/flux-iwjuLe/local-0
log-critical-level                    

## flux module

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Managing Flux extension modules
</div>

Services within a Flux instance are implemented by modules. To query and manage broker modules, use `flux module`.  Modules that we have already directly interacted with in this tutorial include `resource` (via `flux resource`), `job-ingest` (via `flux` and the Python API) `job-list` (via `flux jobs`) and `job-manager` (via `flux queue`). For the most part, services are implemented by modules of the same name.  In some circumstances, where multiple implementations for a service exist, a module of a different name implements a given service (e.g., in this instance, `sched-fluxion-qmanager` provides the `sched` service and thus `sched.alloc`, but in another instance `sched-simple` might provide the `sched` service).

In [42]:
!flux module list

Module                   Idle  S Service
job-exec                    2  R 
heartbeat                   0  R 
job-list                    2  R 
sched-fluxion-qmanager      2  R sched
content-sqlite              1  R content-backing
resource                    1  R 
job-ingest                  2  R 
content                     1  R 
job-info                    5  R 
kvs-watch                   5  R 
sched-fluxion-resource      2  R 
kvs                         1  R 
cron                     idle  R 
job-manager                 0  R 
barrier                  idle  R 
connector-local             0  R 1000-shell-f3Vw1mYfjD,1000-shell-f3Vw6xW9wD


See the [Flux Management Notebook](02_flux_framework.ipynb) for a small tutorial of unloading and reloading the Fluxion (flux scheduler) modules.

## flux dmesg

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Viewing Flux system messages
</div>


If you need some additional help debugging your Flux setup, you might be interested in `flux dmesg`, which is akin to the [Linux dmesg](https://man7.org/linux/man-pages/man1/dmesg.1.html) but delivers messages for Flux.

In [43]:
!flux dmesg

2024-07-20T22:56:18.760174Z broker.debug[0]: rmmod sched-simple
2024-07-20T22:56:18.760532Z broker.debug[0]: module sched-simple exited
2024-07-20T22:56:18.760597Z resource.debug[0]: aborted 1 resource.acquire(s)
2024-07-20T22:56:18.760615Z job-manager.debug[0]: alloc: stop due to disconnect: Success
2024-07-20T22:56:18.879655Z broker.debug[0]: insmod sched-fluxion-resource
2024-07-20T22:56:18.879925Z sched-fluxion-resource.info[0]: version 0.34.0-38-g0fad5268
2024-07-20T22:56:18.879954Z sched-fluxion-resource.debug[0]: mod_main: resource module starting
2024-07-20T22:56:18.880319Z sched-fluxion-resource.warning[0]: create_reader: allowlist unsupported
2024-07-20T22:56:18.880472Z sched-fluxion-resource.debug[0]: resource graph datastore loaded with rv1exec reader
2024-07-20T22:56:18.880477Z sched-fluxion-resource.info[0]: populate_resource_db: loaded resources from core's resource.acquire
2024-07-20T22:56:18.880519Z sched-fluxion-resource.debug[0]: resource status changed (rankset=[all

### flux start

<div class="alert alert-block" style="background-color:lightgreen">
<span style="font-weight:600">Description:</span> Interactively starting a set of resources
</div>

Sometimes you need to interactively start a set of compute resources. We call this subset a flux instance. You can launch jobs under this instance, akin to how you've done above! In fact, this entire tutorial is started (to give you 4 faux nodes) with a `flux start` command: 

```bash
flux start --test-size=4
```

A Flux instance may be running as the default resource manager on a cluster, a job in a resource manager such as Slurm, LSF, or Flux itself, or as a test instance launched locally. This is really neat because it means you can launch Flux under other resource managers where it is not installed as the system workload manager. You can also execute "one off" commands to it, for example, to see the instance size:

In [4]:
!flux start --test-size=4 flux getattr size

4


When you run `flux start` without a command, it will give you an interactive shell to the instance. When you provide a command (as we do above) it will run it and exit. This is what happens for the command above! The output indicates the number of brokers started successfully. As soon as we get and print the size, we exit.

<br>

# Python Submission API 🐍️
Flux also provides first-class python bindings which can be used to submit jobs programmatically. 

### `flux.job.JobspecV1` to create job specifications

Flux represents work as a standard called the [Jobspec](https://flux-framework.readthedocs.io/projects/flux-rfc/en/latest/spec_25.html). While you could write YAML or JSON, it's much easier to use provided Python functions that take high level metadata (command, resources, etc) to generate them. We can then replicate our previous example of submitting multiple heterogeneous jobs using these Python helpers, and testing that Flux co-schedules them.

In [44]:
import os
import json
import flux
from flux.job import JobspecV1
from flux.job.JobID import JobID

In [45]:
# connect to the running Flux instance
f = flux.Flux()

# Create the Jobspec from a command to run a python script, and specify resources
compute_jobreq = JobspecV1.from_command(
    command=["./compute.py", "120"], num_tasks=1, num_nodes=1, cores_per_task=1
)

# This is the "current working directory" (cwd)
compute_jobreq.cwd = os.path.expanduser("~/flux-tutorial/flux-workflow-examples/job-submit-api/")

# When we submit, we get back the job identifier (JobID)
print(JobID(flux.job.submit(f,compute_jobreq)).f58) # submit and print out the jobid (in f58 format)

ƒ3Vyv4d99Z


Once we create the job, when we submit it in Python we get back a job identifier or jobid. We can then interact with the Flux handle, a connection to Flux, to get information about that job.

### `flux.job.get_job(handle, jobid)` to get job info

In [46]:
# Let's submit again to retrieve (and save) the job identifier
fluxjob = flux.job.submit(f, compute_jobreq)
fluxjobid = JobID(fluxjob.f58)
print(f"🎉️ Hooray, we just submitted {fluxjobid}!\n")

# Here is how to get your info. The first argument is the flux handle, then the jobid
jobinfo = flux.job.get_job(f, fluxjobid)
print(json.dumps(jobinfo))

🎉️ Hooray, we just submitted ƒ3VyvraktF!

{"t_depend": 1721520202.4164655, "t_run": 0.0, "t_cleanup": 0.0, "t_inactive": 0.0, "duration": 0.0, "expiration": 0.0, "name": "compute.py", "cwd": "/home/jovyan/flux-tutorial/flux-workflow-examples/job-submit-api/", "queue": "", "project": "", "bank": "", "ntasks": 1, "ncores": 1, "nnodes": 1, "priority": 16, "ranks": "", "nodelist": "", "success": "", "result": "", "waitstatus": "", "id": 320116914913280, "t_submit": 1721520202.4053006, "t_remaining": 0.0, "state": "SCHED", "username": "jovyan", "userid": 1000, "urgency": 16, "runtime": 0.0, "status": "SCHED", "returncode": "", "dependencies": [], "annotations": {}, "exception": {"occurred": "", "severity": "", "type": "", "note": ""}}


You can now run `flux jobs` to see the jobs that we submit from Python.

In [48]:
!flux jobs -a | grep compute

  ƒ3VyvraktF jovyan   compute.py  F      1      1   0.014s 8660c254a8e5
  ƒ3Vyv4d99Z jovyan   compute.py  F      1      1   0.020s 8660c254a8e5
  ƒ2YnijmLwy jovyan   compute.py  F      1      1   0.031s 8660c254a8e5
  ƒ2YiqfxNdm jovyan   compute.py  F      1      1   0.012s 8660c254a8e5
  ƒ2YYgVHnyV jovyan   compute.py  F      1      1   0.062s 8660c254a8e5
  ƒ2YYE7Ja9d jovyan   compute.py  F      1      1   0.048s 8660c254a8e5


Under the hood, the `Jobspec` class is creating a YAML document that ultimately gets serialized as JSON and sent to Flux for ingestion, validation, queueing, scheduling, and eventually execution.  We can dump the raw JSON jobspec that is submitted, where we can see the exact resources requested and the task set to be executed on those resources.

In [49]:
print(compute_jobreq.dumps(indent=2))

{
  "resources": [
    {
      "type": "node",
      "count": 1,
      "with": [
        {
          "type": "slot",
          "count": 1,
          "with": [
            {
              "type": "core",
              "count": 1
            }
          ],
          "label": "task"
        }
      ]
    }
  ],
  "tasks": [
    {
      "command": [
        "./compute.py",
        "120"
      ],
      "slot": "task",
      "count": {
        "per_slot": 1
      }
    }
  ],
  "attributes": {
    "system": {
      "duration": 0,
      "cwd": "/home/jovyan/flux-tutorial/flux-workflow-examples/job-submit-api/"
    }
  },
  "version": 1
}


### `FluxExecutor` for bulk submission

We can use the FluxExecutor class to submit large numbers of jobs to Flux. This method uses python's `concurrent.futures` interface. Here is an example snippet from [flux-workflow-examples/async-bulk-job-submit/bulksubmit_executor.py](flux-workflow-examples/async-bulk-job-submit/bulksubmit_executor.py).

``` python 
with FluxExecutor() as executor:
    compute_jobspec = JobspecV1.from_command(args.command)
    futures = [executor.submit(compute_jobspec) for _ in range(args.njobs)]
    # wait for the jobid for each job, as a proxy for the job being submitted
    for fut in futures:
    fut.jobid()
    # all jobs submitted - print timings
```

In [51]:
# Submit the FluxExecutor based script.
%run ./flux-workflow-examples/async-bulk-job-submit/bulksubmit_executor.py -n200 /bin/sleep 0

bulksubmit_executor: submitted 200 jobs in 0.16s. 1246.61job/s
bulksubmit_executor: First job finished in about 0.172s
|██████████████████████████████████████████████████████████| 100.0% (298.2 job/s)
bulksubmit_executor: Ran 200 jobs in 0.8s. 249.6 job/s


### `flux.event_watch` to watch events

If you want to get the output of a job (or more generally, stream events) you can do that as follows. Let's submit a quick job, and then look at the output.


In [53]:
# Create the Jobspec from a command to run a python script, and specify resources
jobspec = JobspecV1.from_command(
    command=["echo", "Flux Plumbing 💩️🚽️"], num_tasks=1, num_nodes=1, cores_per_task=1)
jobid = flux.job.submit(f, jobspec)

# Give some time to run and finish
import time
time.sleep(5)

for line in flux.job.event_watch(f, jobid, "guest.output"):
    print(line)

1721520256.00717: header {'version': 1, 'encoding': {'stdout': 'UTF-8', 'stderr': 'UTF-8'}, 'count': {'stdout': 1, 'stderr': 1}, 'options': {}}
1721520256.01083: data {'stream': 'stderr', 'rank': '0', 'eof': True}
1721520256.01085: data {'stream': 'stdout', 'rank': '0', 'data': 'Flux Plumbing 💩️🚽️\n'}
1721520256.01087: data {'stream': 'stdout', 'rank': '0', 'eof': True}


### `flux.job.job_list` to list jobs

Finally, it can be really helpful to get an entire listing of jobs. You can do that as follows. Note that the `job_list` is creating a remote procedure call (rpc) and we call `get` to retrieve the output.

In [ ]:
flux.job.job_list(f).get()

# This concludes Chapter 1! 📗️

In this module, we covered:
1. Submitting jobs with Flux
2. The Flux Hierarchy
3. Flux Process and Job Utilities
4. Python Submission API

To continue with the tutorial, open [Chapter 2](./02_flux_framework.ipynb)